In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the dataset
data = pd.read_csv('/content/sample_data/5gnidd.csv')
# Drop unnecessary columns
unnecessary_columns = ['Unnamed: 0', 'RunTime', 'Min', 'Max', 'sTos', 'dTos', 'sDSb', 'dDSb', 'sHops', 'dHops', 'SrcWin',
                       'DstWin', 'sVid', 'dVid', 'SrcTCPBase', 'DstTCPBase', 'TcpRtt', 'SynAck', 'AckDat']
data = data.drop(unnecessary_columns, axis=1)

# Drop any rows with missing values (NaNs) if present
data = data.dropna()

# Separate features (X) and labels (y)
X = data.drop(['Label', 'Attack Type', 'Attack Tool'], axis=1)
y = data['Attack Type']  # Change the target variable to 'Attack Type'

# Convert categorical columns to numerical using one-hot encoding
categorical_cols = ['Proto', 'Cause', 'State']
X = pd.get_dummies(X, columns=categorical_cols)

# Convert the dataframe to a NumPy array
X = X.to_numpy()

# Convert labels to numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Convert target labels to one-hot encoding
num_classes = len(label_encoder.classes_)
y = tf.keras.utils.to_categorical(y, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 128
epochs = 100
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')

# Make predictions on the test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Convert predicted labels back to original labels
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_test_labels = label_encoder.inverse_transform(y_test_classes)

# Calculate accuracy and other metrics
accuracy = accuracy_score(y_test_labels, y_pred_labels)
print(f'Accuracy: {accuracy:.4f}')

# Generate a classification report
report = classification_report(y_test_labels, y_pred_labels)
print('Classification Report:')
print(report)

# Generate a confusion matrix
confusion = confusion_matrix(y_test_labels, y_pred_labels)
print('Confusion Matrix:')
print(confusion)

# Print the predicted attack types
print("\nPredicted Attack Types:")
for true_attack_type, predicted_attack_type in zip(y_test_labels, y_pred_labels):
    print(f"True: {true_attack_type}\tPredicted: {predicted_attack_type}")

import time
from sklearn.metrics import precision_score, recall_score, f1_score
# ... (Your previous code)

# Get unique attack types
unique_attack_types = np.unique(y_test_labels)

for attack_type in unique_attack_types:
    # Filter test data for the specific attack type
    attack_indices = np.where(y_test_labels == attack_type)[0]
    x_test_attack_type = X_test[attack_indices]
    y_test_attack_type = y_test_labels[attack_indices]

    # Calculate training time
    start_time = time.time()
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)
    end_time = time.time()
    training_time = end_time - start_time

    # Calculate prediction time
    start_time = time.time()
    y_pred_attack_type = model.predict(x_test_attack_type)
    end_time = time.time()
    prediction_time = end_time - start_time

    # Convert predicted labels to original labels
    y_pred_classes_attack_type = np.argmax(y_pred_attack_type, axis=1)
    y_pred_labels_attack_type = label_encoder.inverse_transform(y_pred_classes_attack_type)


    # Make predictions for the specific attack type
    y_pred_attack_type = model.predict(x_test_attack_type)
    y_pred_classes_attack_type = np.argmax(y_pred_attack_type, axis=1)
    y_pred_labels_attack_type = label_encoder.inverse_transform(y_pred_classes_attack_type)

    # Calculate metrics for the specific attack type
    accuracy_attack_type = accuracy_score(y_test_attack_type, y_pred_labels_attack_type)
    precision_attack_type = precision_score(y_test_attack_type, y_pred_labels_attack_type, average='weighted')
    recall_attack_type = recall_score(y_test_attack_type, y_pred_labels_attack_type, average='weighted')
    f1_attack_type = f1_score(y_test_attack_type, y_pred_labels_attack_type, average='weighted')

    print(f'Attack Type: {attack_type}')
    print(f'Accuracy: {accuracy_attack_type:.4f}')
    print(f'Precision: {precision_attack_type:.4f}')
    print(f'Recall: {recall_attack_type:.4f}')
    print(f'F1-Score: {f1_attack_type:.4f}')
    print(f'Training Time: {training_time:.2f} seconds')
    print(f'Prediction Time: {prediction_time:.2f} seconds')







Epoch 1/100
6/6 [==============================] - 1s 37ms/step - loss: 389984.4688 - accuracy: 0.6671 - val_loss: 3820.7759 - val_accuracy: 0.9255
Epoch 2/100
6/6 [==============================] - 0s 9ms/step - loss: 4681.9902 - accuracy: 0.9305 - val_loss: 5534.4624 - val_accuracy: 0.9255
Epoch 3/100
6/6 [==============================] - 0s 11ms/step - loss: 6121.5420 - accuracy: 0.9305 - val_loss: 6176.6631 - val_accuracy: 0.9255
Epoch 4/100
6/6 [==============================] - 0s 11ms/step - loss: 6726.5312 - accuracy: 0.9305 - val_loss: 6320.8628 - val_accuracy: 0.9255
Epoch 5/100
6/6 [==============================] - 0s 7ms/step - loss: 6815.7451 - accuracy: 0.9305 - val_loss: 6168.8076 - val_accuracy: 0.9255
Epoch 6/100
6/6 [==============================] - 0s 9ms/step - loss: 6617.2568 - accuracy: 0.9305 - val_loss: 5837.7056 - val_accuracy: 0.9255
Epoch 7/100
6/6 [==============================] - 0s 8ms/step - loss: 6295.1543 - accuracy: 0.9305 - val_loss: 5355.4922 - v

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


6/6 [==============================] - 0s 20ms/step - loss: 26.7287 - accuracy: 0.9225 - val_loss: 72.3193 - val_accuracy: 0.9362
Epoch 2/100
6/6 [==============================] - 0s 10ms/step - loss: 47.3061 - accuracy: 0.9479 - val_loss: 64.8341 - val_accuracy: 0.9255
Epoch 3/100
6/6 [==============================] - 0s 11ms/step - loss: 41.1003 - accuracy: 0.9278 - val_loss: 59.7482 - val_accuracy: 0.9309
Epoch 4/100
6/6 [==============================] - 0s 13ms/step - loss: 33.2164 - accuracy: 0.9345 - val_loss: 48.3093 - val_accuracy: 0.9362
Epoch 5/100
6/6 [==============================] - 0s 15ms/step - loss: 25.0268 - accuracy: 0.9332 - val_loss: 37.1774 - val_accuracy: 0.9309
Epoch 6/100
6/6 [==============================] - 0s 15ms/step - loss: 34.7952 - accuracy: 0.9318 - val_loss: 50.5893 - val_accuracy: 0.9362
Epoch 7/100
6/6 [==============================] - 0s 15ms/step - loss: 31.9891 - accuracy: 0.9238 - val_loss: 61.1550 - val_accuracy: 0.9255
Epoch 8/100
6/6 [=

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


6/6 [==============================] - 0s 8ms/step - loss: 32.0281 - accuracy: 0.9385 - val_loss: 60.0618 - val_accuracy: 0.9255
Epoch 4/100
6/6 [==============================] - 0s 11ms/step - loss: 56.6131 - accuracy: 0.9358 - val_loss: 149.0904 - val_accuracy: 0.9309
Epoch 5/100
6/6 [==============================] - 0s 11ms/step - loss: 371.6560 - accuracy: 0.9104 - val_loss: 1701.1274 - val_accuracy: 0.9255
Epoch 6/100
6/6 [==============================] - 0s 12ms/step - loss: 2064.4907 - accuracy: 0.9305 - val_loss: 3074.0811 - val_accuracy: 0.9255
Epoch 7/100
6/6 [==============================] - 0s 11ms/step - loss: 2591.2151 - accuracy: 0.9318 - val_loss: 3084.6694 - val_accuracy: 0.9255
Epoch 8/100
6/6 [==============================] - 0s 11ms/step - loss: 1918.8680 - accuracy: 0.9144 - val_loss: 2669.4863 - val_accuracy: 0.9096
Epoch 9/100
6/6 [==============================] - 0s 8ms/step - loss: 1650.7518 - accuracy: 0.9131 - val_loss: 2246.8362 - val_accuracy: 0.9309


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


6/6 [==============================] - 0s 11ms/step - loss: 1143.5353 - accuracy: 0.9091 - val_loss: 2901.4255 - val_accuracy: 0.9255
Epoch 3/100
6/6 [==============================] - 0s 8ms/step - loss: 4527.9136 - accuracy: 0.9305 - val_loss: 5078.3462 - val_accuracy: 0.9255
Epoch 4/100
6/6 [==============================] - 0s 8ms/step - loss: 6213.8936 - accuracy: 0.9318 - val_loss: 5236.2734 - val_accuracy: 0.9255
Epoch 5/100
6/6 [==============================] - 0s 11ms/step - loss: 5976.5874 - accuracy: 0.9291 - val_loss: 4382.2119 - val_accuracy: 0.9255
Epoch 6/100
6/6 [==============================] - 0s 11ms/step - loss: 4928.0410 - accuracy: 0.9278 - val_loss: 3121.2031 - val_accuracy: 0.9149
Epoch 7/100
6/6 [==============================] - 0s 9ms/step - loss: 3378.5374 - accuracy: 0.9265 - val_loss: 1823.6356 - val_accuracy: 0.9202
Epoch 8/100
6/6 [==============================] - 0s 8ms/step - loss: 2297.4207 - accuracy: 0.8730 - val_loss: 1100.4993 - val_accuracy: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
